<a href="https://colab.research.google.com/github/ashnaeldho/EVA5-Phase1-Session14/blob/main/planercnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
!git clone https://github.com/NVlabs/planercnn.git

Cloning into 'planercnn'...
remote: Enumerating objects: 150, done.
remote: Total 150 (delta 0), reused 0 (delta 0), pack-reused 150
Receiving objects: 100% (150/150), 1.33 MiB | 23.81 MiB/s, done.
Resolving deltas: 100% (51/51), done.


In [50]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [51]:
#!pip3 install http://download.pytorch.org/whl/cu92/torch-0.4.1-cp36-cp36m-linux_x86_64.whl
#!pip3 install torchvision

In [52]:
import time
import glob
import torch
import os

from IPython.display import Image, clear_output 
print('PyTorch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

PyTorch 1.6.0+cu101 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15079MB, multi_processor_count=40)


In [53]:
!nvidia-smi

Sat Oct 31 09:19:41 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |     10MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [54]:
!cat /etc/*-release

DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=18.04
DISTRIB_CODENAME=bionic
DISTRIB_DESCRIPTION="Ubuntu 18.04.5 LTS"
NAME="Ubuntu"
VERSION="18.04.5 LTS (Bionic Beaver)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 18.04.5 LTS"
VERSION_ID="18.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=bionic
UBUNTU_CODENAME=bionic


In [55]:
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb

--2020-10-31 09:19:41--  https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.0.24
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.0.24|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?7w_nFhyl_E9bml09W5ZyfvkTgJLNMXk8zZHMZ293P6pr_OQpVumQSBWwS5lVShGgSWCf2M_tUyWSRKWxeqB4-L2NLO3KCOvZLIfuSItKguLWxx3ja_CtmyVXwWUCmkhg1YgeDwto-yL0O_pcrmuErj8oTe05NUMQZ3UsEqOcG_XbpLcI10fnKaMDSt8WpFkRwf91ycai3MpKe_eh8tqV7qx2BQ [following]
--2020-10-31 09:19:41--  https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?7w_nFhyl_E9bml09W5ZyfvkTgJLNMXk8zZHMZ293P6pr_OQpVumQSBWwS5lVShGgSWCf2M_tUyWSRKWxeqB4-L2NLO3KCOvZLI

In [56]:
!ls

anchors						       nms
checkpoint					       options.py
config.py					       planercnn
cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb  plane_utils.py
data_prep					       README.md
datasets					       requirements.txt
evaluate.py					       roialign
evaluate_utils.py				       train_planercnn.py
example_images					       utils.py
LICENSE.md					       visualize_utils.py
models


In [57]:
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb

(Reading database ... 155550 files and directories currently installed.)
Preparing to unpack cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb ...
Unpacking cuda-repo-ubuntu1604-8-0-local-ga2 (8.0.61-1) over (8.0.61-1) ...
Setting up cuda-repo-ubuntu1604-8-0-local-ga2 (8.0.61-1) ...
Note: Check first if apt-key functionality is needed at all - it probably isn't!
OK


In [58]:
!ls /var/

backups  colab			  lib	 lock  mail  run    tmp
cache	 cuda-repo-8-0-local-ga2  local  log   opt   spool


In [59]:
!ls /var/cuda-repo-8-0-local-ga2 | grep .pub

7fa2af80.pub


In [60]:
!apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub

OK


In [61]:
!apt-get update

Get:1 file:/var/cuda-repo-8-0-local-ga2  InRelease
Ign:1 file:/var/cuda-repo-8-0-local-ga2  InRelease
Get:2 file:/var/cuda-repo-8-0-local-ga2  Release [574 B]
Get:2 file:/var/cuda-repo-8-0-local-ga2  Release [574 B]
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ub

In [62]:
!apt-get install cuda-8-0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
cuda-8-0 is already the newest version (8.0.61-1).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


**Installing pytorch with cuda support**

In [63]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2016 NVIDIA Corporation
Built on Tue_Jan_10_13:22:03_CST_2017
Cuda compilation tools, release 8.0, V8.0.61


In [64]:
!pip install torch==0.4.0
#!pip install torchvision

  Using cached https://files.pythonhosted.org/packages/69/43/380514bd9663f1bf708abeb359b8b48d3fabb1c8e95bb3427a980a064c57/torch-0.4.0-cp36-cp36m-manylinux1_x86_64.whl
ERROR: torchvision 0.7.0+cu101 has requirement torch==1.6.0, but you'll have torch 0.4.0 which is incompatible.
ERROR: fastai 1.0.61 has requirement torch>=1.0.0, but you'll have torch 0.4.0 which is incompatible.
  Found existing installation: torch 0.4.1
    Uninstalling torch-0.4.1:
      Successfully uninstalled torch-0.4.1


In [65]:
!git clone https://github.com/UTS-CAS/planercnn_with_adf.git

Cloning into 'planercnn_with_adf'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (138/138), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 138 (delta 18), reused 138 (delta 18), pack-reused 0
Receiving objects: 100% (138/138), 13.31 MiB | 47.31 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [66]:
%cd /content/planercnn_with_adf

/content/planercnn_with_adf


In [67]:
!pip install -r requirements.txt

  Using cached https://files.pythonhosted.org/packages/49/0e/e382bcf1a6ae8225f50b99cc26effa2d4cc6d66975ccf3fa9590efcbedce/torch-0.4.1-cp36-cp36m-manylinux1_x86_64.whl
ERROR: torchvision 0.7.0+cu101 has requirement torch==1.6.0, but you'll have torch 0.4.1 which is incompatible.
ERROR: fastai 1.0.61 has requirement torch>=1.0.0, but you'll have torch 0.4.1 which is incompatible.
  Found existing installation: torch 0.4.0
    Uninstalling torch-0.4.0:
      Successfully uninstalled torch-0.4.0


In [68]:
!ls /usr/local/cuda-8.0

bin  extras   lib64	 libnvvp  nvml	README	 share	targets  version.txt
doc  include  libnsight  LICENSE  nvvm	samples  src	tools


In [69]:
%cd /content/planercnn/


/content/planercnn


In [70]:
!export CUDA_PATH=/usr/local/cuda-8.0/

In [71]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2016 NVIDIA Corporation
Built on Tue_Jan_10_13:22:03_CST_2017
Cuda compilation tools, release 8.0, V8.0.61


In [72]:
cd ../

/content


In [73]:
cd /content/planercnn/nms/src/cuda/

/content/planercnn/nms/src/cuda


In [74]:
!ls

nms_kernel.cu  nms_kernel.cu.o	nms_kernel.h


In [75]:
!sudo apt-get update && \
sudo apt-get install build-essential software-properties-common -y && \
sudo add-apt-repository ppa:ubuntu-toolchain-r/test -y && \
sudo apt-get update && \
sudo apt-get install gcc-5 g++-5 -y && \
sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-5 50 --slave /usr/bin/g++ g++ /usr/bin/g++-5 && \
gcc -v

Get:1 file:/var/cuda-repo-8-0-local-ga2  InRelease
Ign:1 file:/var/cuda-repo-8-0-local-ga2  InRelease
Get:2 file:/var/cuda-repo-8-0-local-ga2  Release [574 B]
Get:2 file:/var/cuda-repo-8-0-local-ga2  Release [574 B]
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:14 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/u

In [76]:
!gcc --version

gcc (Ubuntu 5.5.0-12ubuntu1) 5.5.0 20171010
Copyright (C) 2015 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [77]:
!sudo apt-get install gcc-5 g++-5 g++-5-multilib gfortran-5

Reading package lists... Done
Building dependency tree       
Reading state information... Done
g++-5 is already the newest version (5.5.0-12ubuntu1).
g++-5-multilib is already the newest version (5.5.0-12ubuntu1).
gcc-5 is already the newest version (5.5.0-12ubuntu1).
gfortran-5 is already the newest version (5.5.0-12ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [78]:
!sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-5 50 --slave /usr/bin/g++ g++ /usr/bin/g++-5

In [79]:
!sudo update-alternatives --config gcc

There is only one alternative in link group gcc (providing /usr/bin/gcc): /usr/bin/gcc-5
Nothing to configure.


In [80]:
!gcc --version

gcc (Ubuntu 5.5.0-12ubuntu1) 5.5.0 20171010
Copyright (C) 2015 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [81]:
!nvcc -c -o nms_kernel.cu.o nms_kernel.cu -x cu -Xcompiler -fPIC -arch=sm_60

In [82]:
cd ../../

/content/planercnn/nms


In [83]:
!python build.py

Including CUDA code.
/content/planercnn/nms
generating /tmp/tmpw9yd4rxo/_nms.c
setting the current directory to '/tmp/tmpw9yd4rxo'
running build_ext
building '_nms' extension
creating content
creating content/planercnn
creating content/planercnn/nms
creating content/planercnn/nms/src
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -DWITH_CUDA -I/usr/local/lib/python3.6/dist-packages/torch/utils/ffi/../../lib/include -I/usr/local/lib/python3.6/dist-packages/torch/utils/ffi/../../lib/include/TH -I/usr/local/lib/python3.6/dist-packages/torch/utils/ffi/../../lib/include/THC -I/usr/local/cuda/include -I/usr/include/python3.6m -c _nms.c -o ./_nms.o -std=c99
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -DWITH_CUDA -I/usr/local/lib/python3.6/dist-packages/torch/utils/ffi/.

In [84]:
cd ..

/content/planercnn


In [85]:
cd /content/planercnn/roialign/roi_align/src/cuda/

/content/planercnn/roialign/roi_align/src/cuda


In [86]:
!nvcc -c -o crop_and_resize_kernel.cu.o crop_and_resize_kernel.cu -x cu -Xcompiler -fPIC -arch=sm_60

In [87]:
cd ../../

/content/planercnn/roialign/roi_align


In [88]:
!python build.py

Including CUDA code.
/content/planercnn/roialign/roi_align
generating /tmp/tmpvykuexaw/_crop_and_resize.c
setting the current directory to '/tmp/tmpvykuexaw'
running build_ext
building '_crop_and_resize' extension
creating content
creating content/planercnn
creating content/planercnn/roialign
creating content/planercnn/roialign/roi_align
creating content/planercnn/roialign/roi_align/src
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -DWITH_CUDA -I/usr/local/lib/python3.6/dist-packages/torch/utils/ffi/../../lib/include -I/usr/local/lib/python3.6/dist-packages/torch/utils/ffi/../../lib/include/TH -I/usr/local/lib/python3.6/dist-packages/torch/utils/ffi/../../lib/include/THC -I/usr/local/cuda/include -I/usr/include/python3.6m -c _crop_and_resize.c -o ./_crop_and_resize.o -std=c99 -fopenmp -std=c99
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-s

In [89]:
cd ../../

/content/planercnn


In [90]:
#!python evaluate.py --methods=f --suffix=warping_refine --dataset=inference --customDataFolder=example_images

In [91]:
####################################################33333

In [92]:
#ls

In [93]:
#cd planercnn/

In [94]:
#!ls

In [95]:
#!pwd
'''
%cd /content/planercnn/nms/src/cuda/
!nvcc -c -o nms_kernel.cu.o nms_kernel.cu -x cu -Xcompiler -fPIC -arch=sm_61
%cd ../../
!python build.py
%cd ../
'''

'\n%cd /content/planercnn/nms/src/cuda/\n!nvcc -c -o nms_kernel.cu.o nms_kernel.cu -x cu -Xcompiler -fPIC -arch=sm_61\n%cd ../../\n!python build.py\n%cd ../\n'

In [96]:
#!python evaluate.py --methods=f --suffix=warping_refine --dataset=inference --customDataFolder=images